In [49]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_md')

# New Business Details

In [58]:
business_category = 'Beer' # Would be entered by the user, who is planning to set up a new Restaurant

# The list of all the preferences that the user wants in his restaurant
business_preferences = {
    'Alcohol': 'beer_and_wine',
    'BikeParking':0,
    'BusinessAcceptsCreditCards':1,
    'Caters':0,
    'GoodForKids':0,
    'HasTV':1,
    'NoiseLevel':'loud',
    'OutdoorSeating':1,
    'RestaurantsAttire':'formal',
    'RestaurantsDelivery':1,
    'RestaurantsGoodForGroups':0,
    'RestaurantsPriceRange2':2,
    'RestaurantsReservations':1,
    'RestaurantsTableService':0,
    'RestaurantsTakeOut':1,
    'WheelchairAccessible':0,
    'WiFi':'paid'
}


In [59]:
business_predicted_rating = 4 # This value will come from phase 1 of the project

# Suggestion for Improvement

In [60]:
# Categories on which the current solution suggests
suggestion_category = ['Alcohol','BikeParking','BusinessAcceptsCreditCards','Caters','GoodForKids','HasTV','NoiseLevel','OutdoorSeating','RestaurantsAttire','RestaurantsDelivery','RestaurantsGoodForGroups','RestaurantsPriceRange2','RestaurantsReservations','RestaurantsTableService','RestaurantsTakeOut','WheelchairAccessible','WiFi']
suggestion_category

['Alcohol',
 'BikeParking',
 'BusinessAcceptsCreditCards',
 'Caters',
 'GoodForKids',
 'HasTV',
 'NoiseLevel',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'WheelchairAccessible',
 'WiFi']

In [61]:
# Read the id of the business entered by the user 
df_catmap = pd.read_csv('yelp_dataset\\catmap_Regional.csv', header=None)
cat_map = {}
for key,val in zip(df_catmap[0], df_catmap[1]):
    cat_map[key] = val
#business_category_idx = cat_map[business_category]
df = pd.read_csv('yelp_dataset\\businesses_withCluster_Regional.csv')

In [62]:
from numpy import argmax
category_similarity_vector = []
cat_map_keys = list(cat_map.keys())
token1 = nlp(business_category)
for category in cat_map_keys:
    category_similarity_vector.append(token1.similarity(nlp(category)))
most_similar_business_category = cat_map_keys[argmax(category_similarity_vector)]
print("Your business is most similar to: ", most_similar_business_category)
business_category_idx = cat_map[most_similar_business_category]

Your business is most similar to:  Irish


In [63]:
all_businesses_in_category = df[df['catRegion_label'] == business_category_idx]
k = int(len(all_businesses_in_category) * 0.1) #Take top 10 percent of the ratings

#consider only ratings that are greater than the predicted business rating
all_better_businesses_in_category = all_businesses_in_category[all_businesses_in_category['stars'] > business_predicted_rating]
best_k = all_better_businesses_in_category.nlargest(k,'stars',keep='first')

In [64]:
best_k[['stars'] + suggestion_category]

,stars,Alcohol,BikeParking,BusinessAcceptsCreditCards,Caters,GoodForKids,HasTV,NoiseLevel,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,WheelchairAccessible,WiFi
9967,4.5,full_bar,NaN,1.0,NaN,1.0,1.0,average,0.0,casual,NaN,1.0,1.0,0.0,1.0,1.0,NaN,free
30823,4.5,full_bar,0.0,1.0,0.0,0.0,1.0,average,1.0,casual,0.0,1.0,2.0,1.0,1.0,1.0,NaN,free
31387,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36353,4.5,full_bar,NaN,1.0,NaN,0.0,NaN,NaN,1.0,casual,0.0,1.0,2.0,0.0,1.0,0.0,0.0,NaN
39793,4.5,full_bar,0.0,1.0,NaN,0.0,1.0,average,1.0,casual,0.0,1.0,2.0,0.0,0.0,0.0,NaN,no
54776,4.5,full_bar,1.0,1.0,NaN,1.0,1.0,NaN,1.0,casual,0.0,1.0,2.0,1.0,NaN,1.0,NaN,free
66057,4.5,full_bar,1.0,1.0,1.0,0.0,1.0,average,0.0,casual,0.0,1.0,2.0,0.0,1.0,1.0,1.0,free


In [65]:
if(len(best_k) == 0):
    print("You are one of the best in your Category!!")

else:
    best_output = {}
    for category in suggestion_category:
        best_output[category] = best_k.groupby(category)['stars'].sum().reset_index().sort_values(category,ascending=False).iloc[0][category]
    diff = {}
    for key, value in best_output.items():
        if(business_preferences[key] != value):
            diff[key] = [business_preferences[key], value]
    print("These are the areas you can improve : ")
    improved_df = pd.DataFrame.from_dict(diff, orient='index', columns=['Current','Improved'])
    print(improved_df)

These are the areas you can improve : 
                                Current  Improved
Alcohol                   beer_and_wine  full_bar
BikeParking                           0         1
Caters                                0         1
GoodForKids                           0         1
NoiseLevel                         loud   average
RestaurantsAttire                formal    casual
RestaurantsDelivery                   1         0
RestaurantsGoodForGroups              0         1
RestaurantsTableService               0         1
WheelchairAccessible                  0         1
WiFi                               paid        no
